In [ ]:
# Libraries

# EDA 
import numpy as np
import  pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# -Pre-Processing 
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

# -Model Selection
from sklearn.model_selection import train_test_split


# -Algorithms
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# Model validation
from sklearn.model_selection import cross_validate
from sklearn.model_selection import learning_curve 

#Hyper-parameter Hyper-parameter
from sklearn.model_selection import GridSearchCV

# -Metrics
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, classification_report

%matplotlib inline

In [ ]:

#Import of very relevant data 

course_ratings = pd.read_csv('365_database/365_course_ratings.csv')

student_info = pd.read_csv('365_database/365_student_info.csv')

student_learning = pd.read_csv('365_database/365_student_learning.csv')

student_purchases = pd.read_csv('365_database/365_student_purchases.csv')

# Less relevant data

#course_info = pd.read_csv('365_database/365_course_info.csv')

#exam_info = pd.read_csv('365_database/365_exam_info.csv')

#quiz_info = pd.read_csv('365_database/365_quiz_info.csv')

#student_engagement = pd.read_csv('365_database/365_student_engagement.csv')

#student_exams = pd.read_csv('365_database/365_student_exams.csv')

#student_hub_questions = pd.read_csv('365_database/365_student_hub_questions.csv')

#student_quizzes =pd.read_csv('365_database/365_student_quizzes.csv')


# Work process :

<ol>
    <li>
        Dataset preparation and preprocessing
        <ul>
            <li>Data collection</li>
            <li>Data visualization</li>
            <li>Data selection</li>
            <li>Dataset splitting</li>
            <li>Data Cleaning</li>
            <li>Data formatting</li>
            <li>Data Scaling</li>
        </ul>
    </li>
    <li>Modeling and Validation</li>
    <li>Evaluating the model using  test set</li>
</ol>

# Dataset preparation and preprocessing

## Data Collection

In [ ]:
# A glance at the various data

In [ ]:
course_ratings.head()

In [ ]:
student_info.head()

In [ ]:
student_learning.head()

In [ ]:
student_purchases.head()

In [ ]:
# Merge differents data to build a data set 

In [ ]:
df = pd.merge(student_learning, student_purchases, how='left', on='student_id').merge(student_info, how='left', on='student_id').merge(course_ratings, how='left', on=['student_id', 'course_id'])

In [ ]:
df.head()

In [ ]:
df['purchase'] = df['purchase_type']

df.drop('purchase_type', axis=1, inplace=True)

dic = {'Annual':1, 'Monthly':1, 'Quarterly': 1, np.nan: 0}

df['purchase'] = df['purchase'].map(dic)

df['purchase'] = df['purchase'].astype('int')

In [ ]:
df.head(10)

In [ ]:
df.shape

In [ ]:
# the dataset has 87633 sample, not bad 😋 and 12 columns

In [ ]:
df.isnull().sum()

In [ ]:
# A lot of missing values, this is normal due to the merge 😐

In [ ]:
df.dtypes

##### Remarks

In [ ]:
df.describe().T

##### Remarks

- Minutes watched 
    *50% of students watched videos of more than 10 min
- Rating Course
    *Most of the students give, This is normal, since the quality of teaching is excellent

## Data Visualisation

In [ ]:
counts = df['student_country'].value_counts()

df = df.loc[df['student_country'].isin(counts.index[counts > 10])]

## Data selection

In [ ]:
data = df.copy()
data = df.drop(['purchase_id', 'student_id', 'date_purchased'], axis=1)

In [ ]:
data.head()

In [ ]:
data.isna().sum()

## Data split

In [ ]:
X = data.drop('purchase', axis=1)
y = data['purchase']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=365, stratify=y)

In [ ]:
#check distributions

print(y_train.value_counts(normalize=True), y_test.value_counts(normalize=True))

## Cleaning

1 . Inpute Missing Values
- Inpute student_country by most frequent
- Inputer date_rated by most il same in date_registered
- Inpute course_rating by most freqent 

In [ ]:
#define strategy

# fit x_train and transform x_test data

x_train['date_rated'].fillna(x_train['date_registered'], inplace=True)

x_test['date_rated'].fillna(x_train['date_registered'], inplace=True)

inputer = SimpleImputer(strategy='most_frequent')

fit = inputer.fit_transform(x_train)

x_train = pd.DataFrame(data = fit, columns=inputer.feature_names_in_)
x_test  = pd.DataFrame(data = inputer.transform(x_test), columns=inputer.feature_names_in_)


In [ ]:
x_train.head()

In [ ]:
# We check if we haven't forgotten something

print("Missing values in train data : \n" , x_train.isnull().sum())
print("Missing values in test data : \n", x_train.isnull().sum())

In [ ]:
x_train.head()

In [ ]:
#Converte date_registered, date_watched,date_purchased to date time
cols = ['date_registered', 'date_watched', 'date_rated']
for i in cols:
    x_train[i] = pd.to_datetime(x_train[i])
    x_test[i] = pd.to_datetime(x_test[i])

In [ ]:
x_train.isna().sum()

## Feature Engineering 

<p> I will create a column that will group the time between the date of recording and the date of viewing the course, 
technically  (date_registered - date_watched) </p>

In [ ]:
x_train['period'] = x_train['date_watched'] - x_train['date_registered']
x_test['period'] = x_test['date_watched'] - x_test['date_registered']


x_train['period_bf_rating'] = x_train['date_rated'] - x_train['date_watched']
x_test['period_bf_rating']  = x_test['date_rated'] - x_test['date_watched']


In [ ]:
x_train.head()

In [ ]:
x_train.isnull().sum()

## Encodage

In [ ]:
# We are good now, we can build and evaluate our models

In [ ]:
encoder  = OrdinalEncoder()

label = ['student_country']

x_train[label] = encoder.fit_transform(x_train[label])

x_test[label] = encoder.transform(x_test[label])


In [ ]:
# drop date times columns

In [ ]:
x_train.drop(columns=['date_registered', 'date_rated', 'date_watched'], inplace=True)
x_test.drop(columns=['date_registered', 'date_rated', 'date_watched'], inplace=True)

#
x_train = x_train.astype({'course_id': 'int64','minutes_watched': 'int64', 'course_rating': 'int64', 'period': 'str', 'period_bf_rating': 'str'})
x_test  = x_test.astype({'course_id': 'int64','minutes_watched': 'int64', 'course_rating': 'int64', 'period': 'str', 'period_bf_rating': 'str'})

#
x_train['period'] = np.abs((x_train['period'].apply(lambda x : x.split(' ')[0])).astype('int64'))
x_test['period'] = np.abs((x_test['period'].apply(lambda x : x.split(' ')[0])).astype('int64'))

#
x_train['period_bf_rating'] = np.abs((x_train['period_bf_rating'].apply(lambda x : x.split(' ')[0])).astype('int64'))
x_test['period_bf_rating'] = np.abs((x_test['period_bf_rating'].apply(lambda x : x.split(' ')[0])).astype('int64'))

## Model building

In [ ]:
model = SVC()

In [ ]:
model.fit(x_train, y_train)